In [ ]:
# ✨  MONO Modal-Resonator Bank  (auto-tail) ✨
# -------------------------------------------
%pip install --quiet --upgrade numpy soundfile IPython

import numpy as np, soundfile as sf
from math import pi, cos
from IPython.display import Audio, display

BESSEL_RATIOS = [
    1.000, 1.593, 2.136, 2.650, 3.142,   # m=1 (0,1,2…)
    3.615, 4.072, 4.515, 4.946, 5.368,
    5.783, 6.191, 6.592, 6.988, 7.378,
    7.764, 8.145, 8.521, 8.894, 9.263
]

# ---------- helpers ------------------------------------------------
def modal_frequencies(f1, n):               # swap with Bessel roots later
    ratios = BESSEL_RATIOS[:n]
    return [f1 * r for r in ratios]

def excite(ms=5, fs=48_000):
    n  = int(fs * ms * 1e-3)
    burst = 0.3 * np.random.randn(n)        # -10 dBFS noise
    env   = np.exp(-np.arange(n)/fs * 800)  # 1.25 ms decay
    return burst * env

# ---------- ModalResonator ----------------------------------------
class ModalResonator:
    def __init__(self, f0, t60, fs):
        self.fs  = fs
        self.t60 = t60                     # keep for tail calc
        self._set_coeffs(f0, t60)
        self.y1 = self.y2 = 0.0

    def _set_coeffs(self, f0, t60):
        r  = 10 ** (-3 / (t60 * self.fs))
        w0 = 2 * pi * f0 / self.fs
        self.a1 = -2 * r * cos(w0)
        self.a2 =  r * r
        self.g  = 1.0                      # unity impulse peak

    def process(self, x):
        y = self.g * x - self.a1*self.y1 - self.a2*self.y2
        self.y2, self.y1 = self.y1, y
        return y

# ---------- ResonatorBank -----------------------------------------
class ResonatorBank:
    def __init__(self, f1=140, n=16, t60=3.0, slope=-9,
                 fs=48_000, ring_gain=0.25):
        self.ring_gain = ring_gain
        self.modes = []
        for f in modal_frequencies(f1, n):
            t = t60 * 10**((slope/6) * np.log2(f/f1))
            self.modes.append(ModalResonator(f, t, fs))

    # — predicted seconds until signal < cut_dB below peak —
    def tail_seconds(self, cut_dB=90, safety=1.05):
        t60_max = max(m.t60 for m in self.modes)
        return safety * (cut_dB / 60) * t60_max

    def process(self, x):
        acc = 0.0
        for m in self.modes:
            acc += m.process(x)
        return self.ring_gain * acc

# ---------- render -------------------------------------------------
fs          = 48_000
dry_mix     = 0.03
t60_base    = 10.0
ring_gain   = 0.25
cut_dB      = 90                         # render to -90 dB

# build bank first so we can ask it for tail length
bank = ResonatorBank(f1=140, n=16, t60=t60_base,
                     slope=-9, fs=fs, ring_gain=ring_gain)

tail_secs = bank.tail_seconds(cut_dB=cut_dB)          # math-based tail
inp       = np.concatenate([excite(5, fs),
                            np.zeros(int(fs * tail_secs))])

out = np.zeros_like(inp)
for n, x in enumerate(inp):
    out[n] = dry_mix * x + bank.process(x)

# ---------- normalise & save --------------------------------------
if (pk := np.max(np.abs(out))) > 0:
    out /= pk
sf.write("drum_ping_mono.wav",
         np.clip(out, -1, 1),
         fs, subtype="PCM_16")

print(f"Wrote drum_ping_mono.wav  •  {len(out)/fs:.2f} s (tail {tail_secs:.2f}s)")
display(Audio("drum_ping_mono.wav"))

Note: you may need to restart the kernel to use updated packages.
Wrote drum_ping_mono.wav  •  15.76 s (tail 15.75s)
